In [1]:
import pandas as pd
import json
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector


In [2]:
with open("/home/jovyan/work/config/snowflake_config.json") as f:
    creds = json.load(f)

In [3]:
conn = snowflake.connector.connect(
    user=creds["user"],
    password=creds["password"],
    account=creds["account"],
    warehouse=creds["warehouse"],
    database=creds["database"],
    schema=creds["schema"],
    role=creds["role"]
)

cs = conn.cursor()

In [4]:
# Create table if it doesn’t exist
cs.execute("""
CREATE TABLE IF NOT EXISTS FACT_LTV (
    CUSTOMER_ID STRING,
    LIFETIME_VALUE FLOAT
)
""")

In [5]:
df = pd.read_parquet("/home/jovyan/work/staging/fact_ltv.parquet")

In [6]:
print("DataFrame columns:", df.columns)

DataFrame columns: Index(['customer_id', 'lifetime_value'], dtype='object')


In [7]:
df.columns = [col.upper() for col in df.columns]

In [8]:
print("DataFrame columns:", df.columns)

DataFrame columns: Index(['CUSTOMER_ID', 'LIFETIME_VALUE'], dtype='object')


In [9]:
print("📤 Uploading to Snowflake table FACT_LTV...")
success, nchunks, nrows, _ = write_pandas(conn, df, 'FACT_LTV')

📤 Uploading to Snowflake table FACT_LTV...


In [10]:
if success:
    print(f"✅ Upload successful: {nrows} rows uploaded in {nchunks} chunks.")
else:
    print("❌ Upload failed.")

✅ Upload successful: 99 rows uploaded in 1 chunks.
